In [1]:
%cd /home/dk/Desktop/projects/video_membership_fastapi/

/home/dk/Desktop/projects/video_membership_fastapi


In [2]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [3]:
db.get_session()
sync_table(User)

In [4]:
post_data = {
    "email": "abc@abc.com",
    "password": "abc123"
}

In [5]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [6]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [7]:
UserLoginSchema(**post_data) # email="abc@abc", password="abc123"

UserLoginSchema(email='abc@abc.com', password=SecretStr('**********'))

In [8]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
        
    
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
        
    @validator("password_confirm")
    def passwords_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Passwords do not match")
        return v

In [10]:
data = UserSignupSchema(email='abc@abc.com', password='abc123', password_confirm='abc123')

In [12]:
data.json()

'{"email": "abc@abc.com", "password": "**********", "password_confirm": "**********"}'

In [11]:
data.dict()['password'].get_secret_value()

'abc123'

In [14]:
dir(data.dict()['password'])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_secret_value',
 'display',
 'get_secret_value',
 'max_length',
 'min_length',
 'validate']